In [ ]:
!pip install transformers datasets scikit-learn torch

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Charger les données
data_path = '../data/train_data.txt'
df = pd.read_csv(data_path, sep='\t', header=None, names=['label', 'text'])

# Nettoyer
df['text'] = df['text'].fillna('')
df = df[df['text'].str.len() > 50]  # Garder seulement textes > 50 caractères

# Mapper les labels en IDs
label_map = {
    'identite': 0,
    'releve_bancaire': 1,
    'facture_electricite': 2,
    'facture_eau': 3,
    'document_employeur': 4
}
df['label_id'] = df['label'].map(label_map)

# Split train/val (80/20)
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_id'], random_state=42)

print(f"Train: {len(train_df)} exemples")
print(f"Val: {len(val_df)} exemples")
print(f"\nRépartition:")
print(train_df['label'].value_counts())

Train: 68 exemples
Val: 17 exemples

Répartition:
label
facture_electricite    19
facture_eau            15
document_employeur     12
identite               11
releve_bancaire        11
Name: count, dtype: int64


In [ ]:
import random

def augment_text(text, num_augmentations=2):
    """Crée des variations du texte pour augmenter le dataset"""
    augmented = [text]

    words = text.split()

    for _ in range(num_augmentations):
        # Variation 1: Supprimer 10% des mots aléatoirement
        if len(words) > 10:
            new_words = [w for w in words if random.random() > 0.1]
            augmented.append(' '.join(new_words))

        # Variation 2: Mélanger légèrement l'ordre (swap de mots adjacents)
        shuffled = words.copy()
        for i in range(0, len(shuffled)-1, 2):
            if random.random() > 0.7:
                shuffled[i], shuffled[i+1] = shuffled[i+1], shuffled[i]
        augmented.append(' '.join(shuffled))

    return augmented

# Augmenter le dataset
print(" Augmentation du dataset...")
augmented_data = []

for idx, row in train_df.iterrows():
    augmented_texts = augment_text(row['text'], num_augmentations=3)
    for aug_text in augmented_texts:
        augmented_data.append({
            'text': aug_text,
            'label': row['label'],
            'label_id': row['label_id']
        })

train_df_augmented = pd.DataFrame(augmented_data)
print(f"Dataset augmenté: {len(train_df)} → {len(train_df_augmented)} exemples")

In [ ]:
from transformers import CamembertTokenizer
from torch.utils.data import Dataset

tokenizer = CamembertTokenizer.from_pretrained('camembert-base')

class DocumentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

import torch

train_dataset = DocumentDataset(
    train_df_augmented['text'].tolist(),  
    train_df_augmented['label_id'].tolist(),  
    tokenizer
)

val_dataset = DocumentDataset(
    val_df['text'].tolist(),
    val_df['label_id'].tolist(),
    tokenizer
)

print("Datasets créés")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

✅ Datasets créés


In [ ]:
from transformers import CamembertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model = CamembertForSequenceClassification.from_pretrained(
    'camembert-base',
    num_labels=5
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f" Modèle chargé sur {device}")

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Modèle chargé sur cuda


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='/content/results',
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='/content/logs',
    logging_steps=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_total_limit=2,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("✅ Trainer configuré avec paramètres optimisés")

✅ Trainer configuré avec paramètres optimisés


In [ ]:
print("🚀 Début du fine-tuning...")
trainer.train()
print("✅ Fine-tuning terminé !")

🚀 Début du fine-tuning...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.593400,1.583295,0.294118,0.133690,0.086505,0.294118
2,1.530700,1.534754,0.294118,0.133690,0.086505,0.294118
3,1.534500,1.455078,0.352941,0.228291,0.268382,0.352941
4,1.389600,1.322208,0.764706,0.739496,0.869281,0.764706
5,1.214600,1.139275,0.941176,0.939649,0.950980,0.941176
6,1.050000,0.963350,0.941176,0.941176,0.952941,0.941176
7,0.847900,0.781078,0.941176,0.941176,0.952941,0.941176
8,0.651500,0.619543,0.941176,0.941176,0.952941,0.941176
9,0.536200,0.638255,0.764706,0.680672,0.633987,0.764706
10,0.469100,0.403327,0.941176,0.941176,0.952941,0.941176


✅ Fine-tuning terminé !


In [ ]:
# Évaluation finale
results = trainer.evaluate()
print("\n📊 Résultats finaux:")
for key, value in results.items():
    print(f"  {key}: {value:.4f}")


📊 Résultats finaux:
  eval_loss: 0.9634
  eval_accuracy: 0.9412
  eval_f1: 0.9412
  eval_precision: 0.9529
  eval_recall: 0.9412
  eval_runtime: 0.2369
  eval_samples_per_second: 71.7720
  eval_steps_per_second: 21.1090
  epoch: 10.0000


In [ ]:
# Sauvegarder le modèle et le tokenizer
model.save_pretrained('../../models/camembert_finetuned')
tokenizer.save_pretrained('../../models/camembert_finetuned')

print("Modèle sauvegardé")

✅ Modèle sauvegardé dans /content/camembert_finetuned
